In [1]:
pip install PyMuPDF pdfplumber pandas xlsxwriter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 574.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 721.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 22.5 MB/s eta 0:00:00


In [3]:
import os
import zipfile
import fitz  # PyMuPDF
import pdfplumber
import pandas as pd
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract_text_pymupdf(pdf_path):
    """Extract text from a PDF file using PyMuPDF."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        logging.error(f"Error extracting text from {pdf_path}: {e}")
        return ""

def extract_invoice_details(text):
    """Extract invoice details from text using regex."""
    details = {
        "Invoice no": "",
        "Date": "",
        "Order ID": "",
        "Sold By": "",
        "DL Number": "",
        "FSSAI License No": "",
        "GST": "",
        "CIN": "",
        "Registered Address": "",
        "Premise Address": "",
        "Patient Name": "",
        "Address": "",
        "Place of supply": "",
        "Contact": ""
    }

    regex_patterns = {
        "Invoice no": r"Invoice no\.\s*:\s*(\S+)",
        "Date": r"Date\s*:\s*(\S+)",
        "Order ID": r"Order ID\s*:\s*(\S+)",
        "Sold By": r"Sold By\s*\n([^\n]+)",
        "DL Number": r"DL Number:\s*([^\n]+)",
        "FSSAI License No": r"FSSAI License No:\s*(\S+)",
        "GST": r"GST:\s*(\S+)",
        "CIN": r"CIN:\s*(\S+)",
        "Registered Address": r"Registered Address:\s*([^\n]+)",
        "Premise Address": r"Premise Address:\s*([^\n]+)",
        "Patient Name": r"Patient Name:\s*([^\n]+)",
        "Address": r"Address:\s*([^\n]+)",
        "Place of supply": r"Place of supply:\s*([^\n]+)",
        "Contact": r"Contact:\s*(\S+)"
    }

    for key, pattern in regex_patterns.items():
        match = re.search(pattern, text)
        if match:
            details[key] = match.group(1).strip()

    return details

def extract_tables_pdfplumber(pdf_path):
    """Extract tables from a PDF file using pdfplumber."""
    items_table = []
    gst_table = []
    summary_dict = {
        "TOTAL QUANTITY": "",
        "GROSS AMOUNT": "",
        "SHIPPING & VAS CHARGES": "",
        "DISCOUNT AMOUNT": "",
        "BILL AMOUNT": "",
        "ROUND OFF": "",
        "PAYABLE AMOUNT": ""
    }

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_tables = page.extract_tables()
                for table in page_tables:
                    if table and any("GST" in col for col in table[0]) and any("Taxable Amt" in col for col in table[0]):
                        logging.info(f"Identified GST table on page {page.page_number}")
                        header = table[0]
                        for row in table[1:]:
                            row_dict = dict(zip(header, row))
                            gst_table.append(row_dict)
                    else:
                        logging.info(f"Identified Items table on page {page.page_number}")
                        header = table[0]
                        for row in table[1:]:
                            row_dict = dict(zip(header, row))
                            items_table.append(row_dict)

                # Extract summary information from the text
                text = page.extract_text()
                for key in summary_dict.keys():
                    match = re.search(rf"{key}\s*:\s*([^\n]+)", text)
                    if match:
                        summary_dict[key] = match.group(1).strip()

    except Exception as e:
        logging.error(f"Error extracting tables from {pdf_path}: {e}")

    return items_table, gst_table, summary_dict

def process_zip(zip_path, save_dir, excel_file):
    """Process each PDF in the ZIP file and save extracted data to a single Excel file with multiple sheets."""
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            pdf_files = [file for file in zip_ref.namelist() if file.endswith('.pdf')]

            all_invoice_details = []
            all_items_table = []
            all_gst_table = []
            all_summary_table = []

            for pdf_file in pdf_files:
                file_dir = os.path.join(save_dir, os.path.dirname(pdf_file))
                os.makedirs(file_dir, exist_ok=True)

                with zip_ref.open(pdf_file) as file:
                    pdf_path = os.path.join(save_dir, pdf_file)
                    with open(pdf_path, 'wb') as f:
                        f.write(file.read())

                    logging.info(f"Processing {pdf_file}...")
                    text = extract_text_pymupdf(pdf_path)
                    invoice_details = extract_invoice_details(text)
                    items_table, gst_table, summary_dict = extract_tables_pdfplumber(pdf_path)

                    all_invoice_details.append(invoice_details)
                    all_items_table.extend(items_table)
                    all_gst_table.extend(gst_table)
                    all_summary_table.append(summary_dict)

            df_invoice_details = pd.DataFrame(all_invoice_details)
            df_invoice_details.columns = [
                "Invoice no", "Date", "Order ID", "Sold By", "DL Number",
                "FSSAI License No", "GST", "CIN", "Registered Address",
                "Premise Address", "Patient Name", "Address", "Place of supply",
                "Contact"
            ]

            df_items_table = pd.DataFrame(all_items_table)
            df_gst_table = pd.DataFrame(all_gst_table)
            df_summary_table = pd.DataFrame(all_summary_table)

            with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                df_invoice_details.to_excel(writer, sheet_name='Invoice Details', index=False)
                df_items_table.to_excel(writer, sheet_name='Items', index=False)
                df_gst_table.to_excel(writer, sheet_name='GST Details', index=False)
                df_summary_table.to_excel(writer, sheet_name='Summary', index=False)

            logging.info(f"All extracted data has been saved to {excel_file}.")

    except Exception as e:
        logging.error(f"Error processing ZIP file {zip_path}: {e}")

# Paths
zip_path = '/content/one mg-20240718T110057Z-001.zip'
save_dir = './1MG/'
os.makedirs(save_dir, exist_ok=True)
excel_file = os.path.join(save_dir, '1mg_invoices.xlsx')

# Process the ZIP file
process_zip(zip_path, save_dir, excel_file)
